### Elastic Net

In [2]:
# Importe de bibliotecas e dados
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

# Carregar o arquivo completo
df = pd.read_excel('../baseDeDados/dataBaseTratada.xlsx')
dfTeste = pd.read_excel('../baseDeDados/dataBaseTeste.xlsx')

In [3]:
# Criando novas colunas
df['order_weekday'] = pd.to_datetime(df['order_purchase_timestamp']).dt.dayofweek
df['is_sao_paulo_estado_comprador'] = (df['customer_state'] == 'sao paulo').astype(int)
df['is_sao_paulo_cidade_comprador'] = (df['customer_city'] == 'sao paulo').astype(int)
df['is_sao_paulo_estado_vendedor'] = (df['seller_state'] == 'SP').astype(int)
df['is_sao_paulo_cidade_vendedor'] = (df['seller_city'] == 'sao paulo').astype(int)
df['is_credit_card'] = (df['payment_type'] == 'credit_card').astype(int)

In [23]:
# Selecionar colunas relevantes
selected_columns = ['delivery_time', 'customer_zip_code_prefix', 'is_sao_paulo_estado_comprador', 'is_sao_paulo_cidade_comprador', 'is_sao_paulo_cidade_vendedor', 'is_sao_paulo_estado_vendedor', 'is_credit_card', 'Distancia', 'Latitude_Vendedor', 'Latitude_Comprador', 'Longitude_Comprador', 'Longitude_Vendedor']
data_selected = df[selected_columns]

# Separar as variáveis independentes (features) e a variável dependente (target)
X = data_selected.drop('delivery_time', axis=1)
y = data_selected['delivery_time']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Construir e treinar o modelo Elastic Net
model = ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=42)
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 58.00053674434879
R-squared: 0.12119343009843797


In [17]:
# Criando novas colunas para o dfKaggle
dfTeste['is_sao_paulo_estado_comprador'] = (dfTeste['customer_state'] == 'sao paulo').astype(int)
dfTeste['is_sao_paulo_cidade_comprador'] = (dfTeste['customer_city'] == 'sao paulo').astype(int)
dfTeste['is_sao_paulo_estado_vendedor'] = (dfTeste['seller_state'] == 'SP').astype(int)
dfTeste['is_sao_paulo_cidade_vendedor'] = (dfTeste['seller_city'] == 'sao paulo').astype(int)
dfTeste['is_credit_card'] = (dfTeste['payment_type'] == 'credit_card').astype(int)

In [18]:
# Teste na base de dados do Kaggle
X = dfTeste[['product_weight_g', 'customer_zip_code_prefix', 'is_sao_paulo_estado_comprador', 'is_sao_paulo_cidade_comprador', 'is_sao_paulo_cidade_vendedor', 'is_sao_paulo_estado_vendedor', 'is_credit_card', 'Distancia', 'Latitude_Vendedor', 'Latitude_Comprador']]

# Predições no conjunto de teste
y_pred = model.predict(X)

/home/jose/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(


In [19]:
# Criando o dataFrame do Kaggle
dfTeste['previsoes'] = y_pred
dfKaggle = dfTeste[['order_id', 'previsoes']]

In [20]:
dfKaggle

,order_id,previsoes
0,cee3292f46ede6ea1dfabfcb200fcf47,5465.760827
1,50dca53ca33b739bef09e7933e8b380e,209.839371
2,8087ec71e393d4dc6fc48041fe63cd51,24.781648
3,e6b6557ce111de79b31cc857f20ba212,-453.763572
4,0b09c5e4c2512f627190ac55a78c35a3,-4093.554223
...,...,...
28147,81cedc4d2132eb6f512354166e51b187,-186.229359
28148,95811675a89b66659996da5285a021db,-656.754002
28149,25bd2ad9d282f59e4e6a048ef9bf214f,1263.008180
28150,9f9ac312604229686c04260955495c70,109.011573


In [21]:
# Exportar o DataFrame para um arquivo CSV
dfKaggle.to_csv('../resultadoPrevisoes/previsaoElasticNet.csv', index=False)